# Automatically Create Views in Cloudability

The backbone of Cloudability is views, but their creation on mass can become time consuming.

This script is the answer, choose any business dimension and it will automatically create a view for each value. 

1. Enter token
2. Choose business mapping you want to replicate in views, if your business mapping has 100 values then you will get 100 individual views
3. Choose match operator - default is equals (==) but contains (=@) can also be valuable
4. Decide if you want a prefex in your view to give the audiance some context, for example "Env: Dev" or "Dept: Engineering" - useful for hierarchies
5. Run script in your chosen python CLI (i use JupyterLab)

More information about the API: https://developers.cloudability.com/docs/views-end-point

More information about views: https://support.cloudability.com/hc/en-us/articles/204303708-Views

In [ ]:
###Enter details below

#enter CLDY token
token = "example token"

#Which business mapping do you want to use to create the views?
#use the order of business mappings presented in cloudabiity
bus_mapping_number = 4

#!=@, =@, !=, ==, >=, <=, >, <
Operator = "=="

#Enter prefix e.g. "" or "Env: "
view_prefix = "Env: "

In [ ]:
import re
import pandas as pd
import requests 
import base64
import json
from pandas.io.json import json_normalize
pd.set_option('display.max_colwidth', -1)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

########--Variables--########

url ='https://api.cloudability.com/v3/business-mappings/'+ str(bus_mapping_number) + ''

#encode token for html
token2 = token+":"
message_bytes = token2.encode('ascii')
base64_bytes = base64.b64encode(message_bytes)
base64_message = base64_bytes.decode('ascii')
#API details
headers = {
  'Content-Type': 'application/json',
  'Authorization': 'Basic '+base64_message
}

########--Functions--########

#query cloudability, returns dataframe (table)
def Query_Cloudability(url, headers, payload):
    response = requests.request("GET", url, headers=headers, data = payload)
    print("response code data pull: "+str(response.status_code))
    if  "forecast" in url:
        df = json_normalize(json.loads(response.text)["result"])[["parameters.monthsBack", "parameters.monthsForward", "forecastTotal", "currentEstimate"]]
    elif "business-mappings" in url:
        df = json_normalize(json.loads(response.text)["result"]["statements"])
    elif "v3" in url:
        df = json_normalize(response.json(), record_path=['result'])
    else:
        df = json_normalize(response.json())
    return df

#clean strings (ReplaceRegex function)
def cleanstrings(string):
  cleanr = re.compile('\'')
  cleantext = re.sub(cleanr, '', str(string))
  return cleantext

########--Code--########

#Perform Query and clean
df = Query_Cloudability(url, headers, '')
df['valueExpression'] = df['valueExpression'].apply(cleanstrings)

#loop that creates a view for each value 
for x in df['valueExpression']:
    #this is meat of the script, the JSON Post where you define the name of the view, who to share it with and what to match on
    payload = '{ \r\n  \"title\": \"'+ view_prefix + x + '\",\r\n  \"sharedWithOrganization\": true,\r\n  \"sharedWithUsers\": [],\r\n  \"filters\": [{\"field\":\"category'+ str(bus_mapping_number) + '\",\"comparator\":\"'+ Operator + '\",\"value\":\"'+ x + '\"}]\r\n}\r\nEOF'
    response = requests.request("POST", "https://api.cloudability.com/v3/views", headers=headers, data = payload)
    print(x+" response code: "+str(response.status_code))
    
print("End")
